In [17]:
from nltk import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from string import punctuation 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pandas as pd
from datetime import datetime
import os
import re
import json
from io import StringIO

english_stopwords = set(stopwords.words('english') + list(punctuation)+
['..','...','....','``','//n'])

In [21]:
with open('jeopardy.json') as j:
   df = json.load(j)

In [22]:
df = pd.read_json('jeopardy.json')
df.to_csv (r'jeopardy.txt', index = False)
df = pd.read_csv('jeopardy.txt')

In [23]:
df['category'] = df['category'].str.lower()
df['answer'] = df['answer'].str.lower()
df['question'] = df['question'].str.lower()
df['round'] = df['round'].str.lower()

In [26]:
df['value'] = df['value'].str.replace(',', '').str.replace('$', '')
df['value'] = pd.to_numeric(df['value'])

/var/folders/n8/39l_ts116j73dwgfdhp4wvz40000gn/T/ipykernel_8358/4263738981.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['value'] = df['value'].str.replace(',', '').str.replace('$', '')


In [27]:
df.head()

,category,air_date,question,value,answer,round,show_number
0,history,2004-12-31,"'for the last 8 years of his life, galileo was...",200.0,copernicus,jeopardy!,4680
1,espn's top 10 all-time athletes,2004-12-31,'no. 2: 1912 olympian; football star at carlis...,200.0,jim thorpe,jeopardy!,4680
2,everybody talks about it...,2004-12-31,'the city of yuma in this state has a record a...,200.0,arizona,jeopardy!,4680
3,the company line,2004-12-31,"'in 1963, live on ""the art linkletter show"", t...",200.0,mcdonald\'s,jeopardy!,4680
4,epitaphs & tributes,2004-12-31,"'signer of the dec. of indep., framer of the c...",200.0,john adams,jeopardy!,4680


In [28]:
len(df)

216930

In [29]:
value_counts = df['value'].sort_values(axis=0).value_counts(sort=False)
value_counts[:108465]

5.0           9
20.0          1
22.0          1
50.0          1
100.0      9029
           ... 
13800.0       1
14000.0       1
14200.0       1
16400.0       1
18000.0       1
Name: value, Length: 145, dtype: int64

In [36]:
half = (len(df)/2)
accumulated_answers=0
dollar_threshold=0
for dollar_amount, value_count in value_counts.items():
    accumulated_answers+=value_count
    if accumulated_answers > half:
        dollar_threshold = dollar_amount
        break;

print('over ' + str(dollar_threshold)+' is a high value question')
print ('*considering '+str(accumulated_answers)+' of '+str(len(df))+' answers')

over 600.0 is a high value question
*considering 119805 of 216930 answers


In [37]:
is_high_value = []
q_and_a       = []
dollar_values = []

lemmatizer = WordNetLemmatizer()

for index,row in df.iterrows():
    lemm_text = [lemmatizer.lemmatize(x) for x in word_tokenize((row['question']+' '+row['answer']).lower()) if x not in english_stopwords]
    q_and_a.append(' '.join(lemm_text))
    if row.value <= 600.0:
        is_high_value.append(0)
    else:
        is_high_value.append(1)
    dollar_values.append(row.value)

TypeError: can only concatenate str (not "float") to str

In [41]:
df_final = pd.DataFrame({"dollar_value":dollar_values,"is_high_value":is_high_value,"q_and_a":q_and_a})

In [42]:
X_train, X_test, y_train, y_test =  (train_test_split(df_final.q_and_a,
                                                     df_final.is_high_value, random_state=1))

In [43]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tf = tfidf_vectorizer.fit_transform(X_train)
X_test_tf = tfidf_vectorizer.transform(X_test)

In [46]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf, y_train)
prediction = naive_bayes.predict(X_test_tf)

In [47]:
print('Accuracy: ', accuracy_score(y_test, prediction))

Accuracy:  0.5848133305209872
